In [ ]:
from typing import Tuple,List
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
def uniform_pdf(x:float)->float:
    return 1 if 0<=x<=1 else 0


def uniform_cdf(x:float)->float:
    if x<0 :return 0
    if  0<=x<=1 :return x
    if x>0: return 1

def normal_pdf(x:float,mu:float=0,sigma:float=1)->float:
    sqr2pi=math.sqrt(2*math.pi)
    return math.exp(-(x-mu)**2/2/sigma**2)/(sqr2pi*sigma)


def normal_cdf(x:float,mu:float=0,sigma:float=1)->float:
    return (1+math.erf((x-mu)/math.sqrt(2)/sigma))/2

def inverse_normal_cdf(p:float,mu:float=0,sigma:float=1,tolerance=0.000000001)->float:
    if mu!=0 or sigma!=1:return mu+sigma*inverse_normal_cdf(p,tolerance=tolerance)
    high=10;low=-10
    while high-low>tolerance:
            mid_z=(high+low)/2
            mid_p=normal_cdf(mid_z)
            if mid_p<p:low=mid_z
            else:high=mid_z

    return mid_z


In [ ]:
xs=[inverse_normal_cdf(x/1000,23.4,1) for x in range(1,999)]
df=pd.DataFrame({'AGE':xs})

In [ ]:
#Comparison of normal and flatfoot pressure value in static trials

S1=[inverse_normal_cdf(x/1000,0.00462,0.00205) for x in range(3,997)]
S2=[inverse_normal_cdf(x/1000,0.00931,0.00261) for x in range(3,997)]
S3=[inverse_normal_cdf(x/1000,0.00342,0.00108) for x in range(3,997)]
S4=[inverse_normal_cdf(x/1000,0.01371,0.00469) for x in range(3,997)]
S5=[inverse_normal_cdf(x/1000,0.00024,0.00059) for x in range(3,997)]

Ndf=pd.DataFrame({'sensor1':S1,'sensor2':S2,'sensor3':S3,'sensor4':S4,'sensor5':S5,'label':0})
Ndf=Ndf[(Ndf['sensor1']>0) & (Ndf['sensor3']>0) &(Ndf['sensor4']>0) & (Ndf['sensor5']>0)]
Ndf.index = np.arange(0, len(Ndf) )

#*******************

s1=[inverse_normal_cdf(x/1000,0.00929,0.00456) for x in range(3,997)]
s2=[inverse_normal_cdf(x/1000,0.00124,0.00347) for x in range(3,997)]
s3=[inverse_normal_cdf(x/1000,0.00088,0.00397) for x in range(3,997)]
s4=[inverse_normal_cdf(x/1000,0.01765,0.00311) for x in range(3,997)]
s5=[inverse_normal_cdf(x/1000,0.00676,0.00477) for x in range(3,997)]

Pdf=pd.DataFrame({'sensor1':s1,'sensor2':s2,'sensor3':s3,'sensor4':s4,'sensor5':s5,'label':1})
Pdf=Pdf[(Pdf['sensor1']>0) & (Pdf['sensor3']>0) &(Pdf['sensor4']>0) & (Pdf['sensor5']>0)]
Pdf.index = np.arange(0, len(Pdf) )

#Compine&shuffle both dataframes

frames=[Pdf,Ndf]
test=pd.concat(frames)
test = test.sample(frac = 1)
test.index = np.arange(0, len(test) )

In [ ]:
test

In [ ]:
train,valid,test=np.split(test,[int(0.6*len(df)),int(0.8*len(df))])

In [ ]:
def scale_dataset(DataFrame,oversample=False):

    x=DataFrame[DataFrame.columns[:-1]].values
    y=DataFrame[DataFrame.columns[-1]].values
    scaler=StandardScaler()
    x=scaler.fit_transform(x)
    if oversample:
      ros=RandomOverSampler()
      x,y=ros.fit_resample(x,y)
    data=np.hstack((x, np.reshape(y,(-1,1))))
    return data,x,y

In [ ]:
train,x_train,y_train=scale_dataset(train,oversample=True)
valid,x_valid,y_valid=scale_dataset(valid,oversample=False)
test,x_test,y_test=scale_dataset(test,oversample=False)

In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=100)
knn_model.fit(x_train,y_train)


KNeighborsClassifier(n_neighbors=100)

In [ ]:
y_pred=knn_model.predict(x_valid)

In [ ]:
y_pred

array([1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1])

In [ ]:
y_valid

array([1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1])

In [ ]:
classification_report(y_valid,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       108
           1       1.00      1.00      1.00        92

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200


In [29]:
knn_model.predict([[-0.40395052,  1.34590515,  0.6002467 ,  0.64826182, -0.8022319]])

array([0])

In [ ]:
test